CNN, Kendisinden önce convolution işlemleri içeren bir derin öğrenme sinir ağıdır. Resimler ve videolar için kullanılır.<br>
Veri setimizde kedi ve köpek resimleri bulunmakta ve CNN'yi kedi ve köpekleri ayırt edebilecek şekilde eğiteceğiz.<br><br>
Genelde resim veri setini koda dönüştürme işleminde her bir sınıfa ait resmin ismi o sınıf ismiyle başlar ve ardından sıra sayısı alır. Örneğin eğitim verimizdeki köpek resimleri dog.1, dog.2, ..., dog.5000 diye gidiyor. Bu resimlerin her birini yükleyip 3 boyutlu diziler oluşturup resim isimlerini label olarak ele aldığımızda her bir resmin sınıfı zaten kendi isminde yazıyor olacaktır ve buna göre işlemler yapılır. Ama burada keras'ın sunduğu bir avantajı kullanacağız, resimlerin ismine değil de o resimlerin bulunduğu klasör isimlerini sınıf olarak kullanacağız bunun için kedi ve köpek resimleri farklı klasörlerde bulunmaktadırlar, böylesi daha kolay.

### Building the CNN

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D # full connection yapar
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense # full connection yapmaz

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
# convolutionn -> max pooling -> flattening -> full connection

# initialising the CNN
classifier = Sequential()

Convolution2D(Number of Feature Maps(filters), (Number of Rows in Filters, Number of Cols in Filters), border_mode, input_shape(dimensions of each channel, //, number of channels))<br>
Feature Maps sayısı genellikle 32 ile başlamalı ve ardışık katmanlarda artarak devam etmelidir 64, 128, 256 ...<br>
border mod resimdeki kenarlıkları nasıl işleyeceğiyle alakalıdır, genellikle "same" uygulanır.<br>
input shape, resim verimizin dönüştürüldüğü format bilgisini ister. Renkli resimler için ilk iki parametre boyutu olur, son parametre ise 3 olur 3 boyutlu olduğundan.<br>
Bizim işlemlerimizde renk bilgisini koruyabilmek için 3 seçiyoruz, çünkü kedi ile köpekler arasında renk farklılıkları vardır. Format olarka da 64x64 seçmek işimize yarayacaktır çünkü daha büyük formatlar çok daha uzun zamanlar gerektirir ve o kadar beklemeyelim şimdi saatlerce. Resmin boyutu ufak olsada bu sınıflandırmada işe yarayacaktır. Yine bunlarıda 32, 64, 128, ... diye seçmeliyiz. Örn bu sınıflandırma için 256x256 seçersek 8 saate yakın sürecektir eğitim işlemi.<br>
Feature map'lerde herhangi bir negatif değer olmadığından emin olmak için katmanlara aktivasyon fonksiyonu eklemeyi de ihmal etmiyoruz, çünkü illa oluyor negatif değerler. Aktivasyon fonksiyonu olarak Rectifier kullanıyoruz yani relu.

In [15]:
# Convolution (input image X feature detector -> feature map)
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = "relu"))

Flattening stepte daha az node - nöron olması için pooling yapıyoruz. Süredek oldukça kazandırırken performanstan bir şey kaybettirmiyor ve yoğun hesaplamaların önüne geçiyoruz. Bilgi kaybının önüne geçebileceğimiz en iyi max pool boyutu 2x2 olarak tecrübe edildiğinden 2x2 seçiyoruz. Kısaca feature map boyutlarımız düşürülüyor.

In [16]:
# Pooling (each feature map X max pooling -> pooled feature map)
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Max pool ile boyutlarını düşürüp kendilerini yarıya bölsek de hala elimizde çok fazla sayıda pooled feature maps olacaktır. Çok boyutlu bir vektörü tek boyuta indirgemek de boyutunu düşürür. E direkt resme flattening uygulasak şu önceki adımları(Conv step, maxpool step) atlasak ne olur? Bu durumda her bir pixel bilgisini kaydetmiş oluruz, ama bu pixellerin etraflarındaki pixellerle ilişkilerini ağa anlatamayız, yani bir bilgi çıkarımı yapılamaz.<br>


In [17]:
# Flattening (nXn boyutlu pooled feature maps X flattening -> bir boyutlu array)
# pfm[[1,2,3],[4,5,6],[7,8,9]] -> pfm[1,2,3,4,5,6,7,8,9]
# pfm1, pfm2, pfm3,...,pfmn -> [pfm1, pfm2, pfm3, ..., pfmn]
classifier.add(Flatten())

Node sayısı olarak giriş ve çıkış nöronlarının sayısı diyorduk ama burada bir sürü pool olduğundan numaralarını hesap etmek zor olacaktır. O yüzden küçük olmayan sabit bir sayı seçiyoruz 128(büyüdükçe eğitim daha çok vakit alır), genelde 100 civarında bir node seçimi yapılır ama 2nin kuvveti olması daha iyi oluyor.

In [18]:
# Full Connection (hidden layer) (flattened pooled feature maps -> are inputs)
classifier.add(Dense(units = 128, activation = "relu"))

# output layer
classifier.add(Dense(units = 1, activation = "sigmoid")) # binary çıkış var sigmoid yeterli

In [19]:
# Compiling the CNN
# iki sınıf var diye binary ce seçtik, eğer ikiden fazla olaydı categorical_crossentropy seçerdik
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

### Fitting the CNN

<b>Image Agumentation Process</b><br>
Eğer bu adım yapılmazsa train accuracysi yüksek olur ama test düşük olur, kısaca overfitting gerçekleşir, bu gerçekleşmesin diye bu adımın yapılması gereklidir. Overfitting nasıl oluşur? Elimizde, ağın bütün özellikler arasındaki ilişkiyi doğru şekilde bulabileceği kadar eğitim resim verisi yoksa, yani aslında az resim varsa ağ bu resimleri ve özellikler-pixeller arasındaki ilişkileri öğrenir ama ve yüksek doğruluk oranı verir ama aslında daha fazla özellik ilişkisi öğrenmesi gerekiyordur, geliştirilmesi lazımdır, bu yüzden de test accuracysi düşük çıkar. Bizim eğitim için 8k, test için 2k resmimiz var ama bu aslında iyi bir ağ eğitmek için iyi bir sayı değildir. Bu durumda işte agumentation yapmak işimizi çözer. Bu agumentation işlemi resimlerimizden bir sürü kümelenmeler oluşturur, bu kümelere yerleştirilmiş rasgele resimlere yine rasgele işlemler uygular, resmi kaydırma, çevirme veya kırpma gibi işlemler uygular bu sayede yeni resimler türetilmiş olur. Bu türetmeler sonucunda eğitim verimiz çeşitlenir, sayısı bir hayli artar ve ağ tıpatıp aynı resim bulamaz. Bu sayede az bir eğitim verisiyle iyi bir ağ kurmak mümkün oluyor.

In [3]:
from keras.preprocessing.image import ImageDataGenerator

# 0-255 olan pixel değerlerini 0-1 arasına sığacak şekilde scale eder
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
training_set = train_datagen.flow_from_directory('data/CNN/training_set',
        target_size=(64, 64), # resim input shape ile aynı
        batch_size=32,
        class_mode='binary') # iki sınıfımız var diye binary

Found 8000 images belonging to 2 classes.


In [5]:
test_set = test_datagen.flow_from_directory('data/CNN/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


In [24]:
# eğitim 1 epoch için 35dk sürdü
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000, # kaç tane eğitim verimiz varsa onu yazıyoruz
        epochs=1, # 1 epoch hiç iyi değil ama öbür türlü de saatler alıyor, normali 25ten başlamalı
        validation_data=test_set,
        validation_steps=2000) # test verisi sayısı
# val'lı ifadeler test verisinin sonuçları

Epoch 1/1
8000/8000 [==============================] - 2172s 272ms/step - loss: 0.4170 - acc: 0.7999 - val_loss: 0.6700 - val_acc: 0.7605


### Modeli İyileştirmek - Daha Derin Bir Ağ

Çıkan sonuç %76'larda. Peki %80'i nasıl geçirebiliriz, yani nasıl daha iyi hale getirebiliriz? Daha derin bir sinir ağı modelleyerek tabiki. Bunu yaparken de Convolution katmanını çoğaltıp bir tane daha fully connected layer yapabiliriz. Bu işlemi overfitting olduğunu gördüğümüz durumlarda da gerçekleştirebiliriz, böylece overfittinge de çözüm olur, test ile train accuracilerini daha derin bir ağ birbirlerine yakınlaştıracaktır. Daha da iyi sonuçlar için resimlerin input_shape'leri büyütülebilir(pixellerden daha fazla bilgi alabilmek için), daha da fazla derin katman eklenebilir.

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Daha derin bir ağ için Convolution katmanını çoğalttık ama ilk katmanda kendisine resim verisi gelirken ikinci katmanda kendisine pool edilmiş resim mapleri geliyor. Bunun için input_shape katmanını kaldırıyoruz çünkü elimizde işlenmiş mapler olacak ve buna gerek yok. Daha da iyi sonuçlar istersek 3. bir Convolution katmanı ekleriz ve mesela onun feature maplerinin boyutunu 32 değil de 64 belirleriz, bu sonuçları iyileştirecek-geliştirecektir.

In [2]:
classifier = Sequential()

classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Convolution2D(32, (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = "relu"))


classifier.add(Dense(units = 1, activation = "sigmoid"))

classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [6]:
# veri alma işlemlerini tekrarlamak yerine sadece ağı başlatacak kodu alıyorum
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000,
        epochs=1,
        validation_data=test_set,
        validation_steps=2000)
# val'lı ifadeler test verisinin sonuçları

Epoch 1/1
8000/8000 [==============================] - 1714s 214ms/step - loss: 0.3846 - acc: 0.8185 - val_loss: 0.6028 - val_acc: 0.7795


İlk ağda train ile test accuracileri arasında %4 fark vardı, şimdikinde ise %2.5, ve accuraciler artmış durumda.

### Predicting a Single Image

In [7]:
import numpy as np
from keras.preprocessing import image

In [22]:
test_image = image.load_img("data/CNN/single_prediction/cat_or_dog_1.jpg", target_size = (64, 64))
test_image = image.img_to_array(test_image)
# resme dördüncü bir boyut eklemek gerekiyor hata almamak için
test_image = np.expand_dims(test_image, axis = 0)

In [24]:
result = classifier.predict(test_image)
print(training_set.class_indices)
print(result[0][0])

{'cats': 0, 'dogs': 1}
1.0
